In [ ]:
import os
import sys
import boto3
import datetime
import pandas as pd

from pathlib import Path
from dotenv import load_dotenv

from shared import load_transactions, download_bucket_folder

load_dotenv()


In [ ]:
START_DATE = '2020-04-01'

local_prefix = './data/'


In [ ]:
# get all the data from S3
download_bucket_folder('archive')
download_bucket_folder('fraud')


In [ ]:
# load the archived data
input_dir = local_prefix + 'archive'

files = [os.path.join(input_dir, f) for f in os.listdir(input_dir)]
tx_df = load_transactions(files)


In [ ]:
# load the fraud data
input_dir = local_prefix + 'fraud'

files = [os.path.join(input_dir, f) for f in os.listdir(input_dir)]
fraud_df = load_transactions(files)


In [ ]:
# sort by TXID
tx_df = tx_df.sort_values('TRANSACTION_ID')
tx_df.reset_index(drop=True, inplace=True)

fraud_df = fraud_df.sort_values('TRANSACTION_ID')
fraud_df.reset_index(drop=True, inplace=True)

# merge fraud data into archived data

for index, row in fraud_df.iterrows():
    idx = tx_df.loc[tx_df['TRANSACTION_ID'] == row['TRANSACTION_ID']]
    if len(idx) > 0:
        tx_df.loc[idx.index, ['TX_FRAUD_SCENARIO']] = row['TX_FRAUD_SCENARIO']
        tx_df.loc[idx.index, ['TX_FRAUD_PREDICTION']] = 1
        tx_df.loc[idx.index, ['TX_FRAUD_PROBABILITY']] = 1.0

# sort chronological
tx_df = tx_df.sort_values('TX_DATETIME')
tx_df.reset_index(drop=True, inplace=True)


In [ ]:
# add columns that make sorting, grouping etc easier
start_date = datetime.datetime.strptime(START_DATE, "%Y-%m-%d")

tx_df['TX_TIME_SECONDS'] = pd.to_numeric(tx_df['TX_DATETIME']) / 1000000000
tx_df['TX_TIME_SECONDS'] = tx_df['TX_TIME_SECONDS'].astype(int)

tx_df['TX_TIME_DAYS'] = tx_df['TX_DATETIME'] - start_date
tx_df['TX_TIME_DAYS'] = tx_df['TX_TIME_DAYS'].apply(lambda x: x.days)


In [ ]:
# create the output dir
DIR_OUTPUT = local_prefix + "training/"

# make sure the dir exist
if not os.path.exists(DIR_OUTPUT):
    os.makedirs(DIR_OUTPUT)


In [ ]:
# save the merged data
end_date = tx_df.TX_DATETIME.max() + datetime.timedelta(days=1)

for day in range(tx_df.TX_TIME_DAYS.max()+1):
    date = start_date + datetime.timedelta(days=day)
    filename_output = date.strftime("%Y-%m-%d")

    tx_day = tx_df[tx_df.TX_TIME_DAYS == day].sort_values('TX_TIME_SECONDS')
    tx_day.to_csv(DIR_OUTPUT + filename_output + '.csv', index=False)
